In [2]:
import pandas as pd
import glob
import numpy as np
%matplotlib inline

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import logging

cdir = os.getcwd()
print(cdir)
os.chdir(cdir + '/train/train')
print(os.getcwd())

C:\Projects\activity-recognition-abc
C:\Projects\activity-recognition-abc\train\train


In [5]:
from tqdm import tqdm_notebook

In [4]:
import tensorflow as tf
tf.get_logger().setLevel(logging.ERROR)
from tfdeterminism import patch
patch()
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)
import random
import numpy as np
os.environ['PYTHONHASHSEED']=str(1997)

random.seed(1997)
np.random.seed(1997)
tf.random.set_seed(1997)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only use the fourth GPU
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


TensorFlow version 2.0.0 has been patched using tfdeterminism version 0.3.0


In [6]:
def process(sensordata, timestamps):
    # process
    
    return sensordata

In [7]:
import os
os.sep

'\\'

In [8]:
# turning tensorflow warnings off

In [9]:
# there are 3 subjects
subject1 = {}
subject2 = {}
subject3 = {}

# we will load every single right arm data, separate based on subject id, do feature extraction, run t-SNE

all_sensors = ['right_arm', 'right_wrist', 'left_hip', 'left_wrist'] #, 'mocap']

data_folder = [f'{sensor}/*.csv' for sensor in all_sensors]

print(data_folder)

files = glob.glob(data_folder[0])

# processing will be done as numpy array

subject1['data'] = []
subject2['data'] = []
subject3['data'] = []
 
# data format : data -> id, process(timeseries, timestamps) : right_arm as numpy array (missing data will be imputed)


# id is generated by concatenating INT(subject+trial)

for f in files:
    #print(f)
    c_sub = {} # current subject
    if f.split(os.sep)[1].split('_')[0] == 'subject1':
        
        c_sub['id'] = int(f.split(os.sep)[1].split('_')[0][-1] + f.split(os.sep)[1].split('_')[2].split('.')[0])
        #print(ra['id'])
        
        for sensor in all_sensors:
            #print(sensor)
            ra = pd.read_csv(sensor + os.sep + f.split(os.sep)[1])
            #print(ra)
            ra.sort_values(by=['timestamp'], inplace = True)
            c_sub[sensor] = {}
            if sensor == 'mocap':
                for axis in mocap_axis:
                    c_sub[sensor][axis] = process(ra[axis], ra['timestamp'])
            else:
                for axis in ['X', 'Y', 'Z']:
                    c_sub[sensor][axis] = process(ra[axis], ra['timestamp'])
                
        subject1['data'].append(c_sub)

            
    elif f.split(os.sep)[1].split('_')[0] == 'subject2':
        
        c_sub['id'] = int(f.split(os.sep)[1].split('_')[0][-1] + f.split(os.sep)[1].split('_')[2].split('.')[0])
        #print(ra['id'])
        
        for sensor in all_sensors:
            ra = pd.read_csv(sensor + os.sep + f.split(os.sep)[1])
            #print(ra)
            ra.sort_values(by=['timestamp'], inplace = True)
            c_sub[sensor] = {}
            if sensor == 'mocap':
                for axis in mocap_axis:
                    c_sub[sensor][axis] = process(ra[axis], ra['timestamp'])
            else:
                for axis in ['X', 'Y', 'Z']:
                    c_sub[sensor][axis] = process(ra[axis], ra['timestamp'])
                
        subject2['data'].append(c_sub)
            
    elif f.split(os.sep)[1].split('_')[0] == 'subject3':
        
        c_sub['id'] = int(f.split(os.sep)[1].split('_')[0][-1] + f.split(os.sep)[1].split('_')[2].split('.')[0])
        #print(ra['id'])
        
        for sensor in all_sensors:
            ra = pd.read_csv(sensor + os.sep + f.split(os.sep)[1])
            #print(ra)
            ra.sort_values(by=['timestamp'], inplace = True)
            c_sub[sensor] = {}
            if sensor == 'mocap':
                for axis in mocap_axis:
                    try:
                        c_sub[sensor][axis] = process(ra[axis], ra['timestamp'])
                    except:
                        c_sub[sensor][axis] = [0]
            else:
                for axis in ['X', 'Y', 'Z']:
                    c_sub[sensor][axis] = process(ra[axis], ra['timestamp'])
        
        subject3['data'].append(c_sub)
    else:
        print('either new subject or a bug')
        
        
labels = pd.read_csv("labels.txt", sep=' ', header=None)
print(labels.head())
labels = labels[0].str.split(",", n=2, expand=True)
labels.columns = ['file_id', 'macro', 'micro'] #give names to the columns
labels.index = labels['file_id'] #use the file id as index to make it searchable by file_id
print(labels.head())

# label generation
subject1['label_mac'] = {}
subject2['label_mac'] = {}
subject3['label_mac'] = {}

subject1['label_mic'] = {}
subject2['label_mic'] = {}
subject3['label_mic'] = {}

for i in range(len(labels)):
    #print(labels.iloc[i]['file_id'])
    #print(labels.iloc[i]['file_id'].split('_')[0][-1])
    tid = int(labels.iloc[i]['file_id'].split('_')[0][-1] + labels.iloc[i]['file_id'].split('_')[-1])
    #print(tid)
    label = labels.iloc[i]['macro']
    label_mic = labels.iloc[i]['micro'].split(',')[:-1]
    #print(label)
    if labels.iloc[i]['file_id'].split('_')[0][-1] == '1':
        subject1['label_mac'][tid] = label
        subject1['label_mic'][tid] = label_mic
    elif labels.iloc[i]['file_id'].split('_')[0][-1] == '2':
        subject2['label_mac'][tid] = label
        subject2['label_mic'][tid] = label_mic
    elif labels.iloc[i]['file_id'].split('_')[0][-1] == '3':
        subject3['label_mac'][tid] = label
        subject3['label_mic'][tid] = label_mic
    else:
        print('some bug')
        
        
# re-formatting dataset for training

X = []
y = []
y_ml = []  # multi-label

for i in range(len(subject1['data'])):
    tid = subject1['data'][i]['id']
    y.append(subject1['label_mac'][tid])
    y_ml.append(subject1['label_mic'][tid])
    # X shape -> [ip1, ip2, ip3, ip4] ip1 = (80, len, channel) -> 4, 80, len

for sensor in all_sensors:
    cs_data = []
    for i in range(len(subject1['data'])):
        sub_data = []
        if sensor == 'mocap':
            for sig in mocap_axis:
                sub_data.append(np.array(subject1['data'][i][sensor][sig]))
        else:
            for sig in ['X', 'Y', 'Z']:
                sub_data.append(np.array(subject1['data'][i][sensor][sig]))
        sub_data = np.array(sub_data)
        # print(sub_data.shape)
        sub_data = np.swapaxes(sub_data, 0, 1)
        cs_data.append(sub_data)
    X.append(cs_data)
    
    
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import numpy as np

label_encoder = LabelEncoder()
vec = label_encoder.fit_transform(y)

y_ohe = to_categorical(vec,len(set(vec)))

from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
y_ml_ohe = mlb.fit_transform(y_ml)

# validatin on subject2, 3

# re-formatting dataset for training

X2 = []
y2 = []
y_ml2 = []

for i in range(len(subject2['data'])):
    tid = subject2['data'][i]['id']
    y2.append(subject2['label_mac'][tid])
    y_ml2.append(subject2['label_mic'][tid])
    # X shape -> [ip1, ip2, ip3, ip4] ip1 = (80, len, channel) -> 4, 80, len

for sensor in all_sensors:
    cs_data = []
    for i in range(len(subject2['data'])):
        sub_data = []
        if sensor == 'mocap':
            for sig in mocap_axis:
                sub_data.append(np.array(subject2['data'][i][sensor][sig]))
        else:
            for sig in ['X', 'Y', 'Z']:
                sub_data.append(np.array(subject2['data'][i][sensor][sig]))
        sub_data = np.array(sub_data)
        # print(sub_data.shape)
        sub_data = np.swapaxes(sub_data, 0, 1)
        cs_data.append(sub_data)
    X2.append(cs_data)
    
label_encoder = LabelEncoder()
vec = label_encoder.fit_transform(y2)

y2_ohe = to_categorical(vec,len(set(vec)))

mlb = MultiLabelBinarizer()
y_ml2_ohe = mlb.fit_transform(y_ml2)

# validatin on subject2, 3

# re-formatting dataset for training

X3 = []
y3 = []
y_ml3 = []

for i in range(len(subject3['data'])):
    tid = subject3['data'][i]['id']
    y3.append(subject3['label_mac'][tid])
    y_ml3.append(subject3['label_mic'][tid])
    # X shape -> [ip1, ip2, ip3, ip4] ip1 = (80, len, channel) -> 4, 80, len

for sensor in all_sensors:
    cs_data = []
    for i in range(len(subject3['data'])):
        sub_data = []
        if sensor == 'mocap':
            for sig in mocap_axis:
                sub_data.append(np.array(subject3['data'][i][sensor][sig]))
        else:
            for sig in ['X', 'Y', 'Z']:
                sub_data.append(np.array(subject3['data'][i][sensor][sig]))
        sub_data = np.array(sub_data)
        # print(sub_data.shape)
        sub_data = np.swapaxes(sub_data, 0, 1)
        cs_data.append(sub_data)
    X3.append(cs_data)
    

label_encoder = LabelEncoder()
vec = label_encoder.fit_transform(y3)

y3_ohe = to_categorical(vec,len(set(vec)))

mlb = MultiLabelBinarizer()
y_ml3_ohe = mlb.fit_transform(y_ml3)

['right_arm/*.csv', 'right_wrist/*.csv', 'left_hip/*.csv', 'left_wrist/*.csv']
                                           0
0           subject2_file_457,sandwich,Take,
1      subject2_file_679,sandwich,Wash,Take,
2        subject2_file_95,sandwich,Cut,Wash,
3  subject2_file_899,sandwich,other,Cut,Put,
4            subject2_file_368,sandwich,Put,
                             file_id     macro           micro
file_id                                                       
subject2_file_457  subject2_file_457  sandwich           Take,
subject2_file_679  subject2_file_679  sandwich      Wash,Take,
subject2_file_95    subject2_file_95  sandwich       Cut,Wash,
subject2_file_899  subject2_file_899  sandwich  other,Cut,Put,
subject2_file_368  subject2_file_368  sandwich            Put,


In [10]:
import tensorflow as tf
from tensorflow.keras.layers import subtract, Conv1D, Dense, LSTM, Bidirectional, Input, GlobalMaxPooling1D, Concatenate, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LeakyReLU

In [11]:
# CNN-head
import tensorflow.keras as keras
def create_model():

    n_sensors = 4
    n_filters = [3, 3, 3, 3] # X, Y, Z


    # make the input layers
    ips = []

    n_f = 0
    for sensor in all_sensors:
        ips.append(Input(shape = (None, n_filters[n_f]), name = sensor + '_ipX'))
        n_f += 1

    print(f'Input tensors: {ips}')
    
    # kernel size = 1 == embedding layer

    convs = []
    for ip in ips:
        conv = Conv1D(filters = 32, kernel_size = 1, padding = 'same', activation = None, name = ip.name.split(':')[0] + '_conv')(ip)
        convs.append(conv)
        
    # batch-normalization
    bns = []
    for conv in convs:
        bn = BatchNormalization()(conv)
        bns.append(bn)

    convs2 = []
    for bn in bns:
        conv2 = Conv1D(filters = 16, kernel_size = 3, dilation_rate = 2, 
                       kernel_regularizer = keras.regularizers.l1_l2(l1=0.01, l2=0.002),
                       bias_regularizer = keras.regularizers.l1_l2(l1=0.01, l2=0.002), 
                       padding = 'same', activation = 'relu')(bn)
        convs2.append(conv2)
        
    # batch-normalization
    bns2 = []
    for conv2 in convs2:
        bn2 = BatchNormalization()(conv2)
        bns2.append(bn2)
        
    # residual
    
    res = []
    cnt_i = 0
    for bn2 in bns2:
        rs = bn2 #Add()([bn2, bns[cnt_i]])
        rs = LeakyReLU(alpha=0.3)(rs)
        cnt_i += 1
        res.append(rs)



    # concatenation

    concat = Concatenate(axis=1, name = 'concat')(res)

    concat_conv = Conv1D(filters = 16, kernel_size = 1, padding = 'same', dilation_rate = 2,
                        kernel_regularizer = keras.regularizers.l1_l2(l1=0.01, l2=0.002),
                        bias_regularizer = keras.regularizers.l1_l2(l1=0.01, l2=0.002),
                        activation = 'relu', name = 'concat_conv')(concat)
    
    bn3 = BatchNormalization()(concat_conv)
    con_conv2 = Conv1D(filters = 32, kernel_size = 3, padding = 'same', activation = None, name = 'con_conv2')(bn3)
    bn4 = BatchNormalization()(con_conv2)
    
    #res2 = Add()([bn4, bn3])
    res2 = LeakyReLU(alpha=0.3)(bn4)
    
    gmp = GlobalMaxPooling1D()(res2)

    # fully convolutional
    simple_conv = Model(ips, gmp)
    
    return simple_conv

In [12]:
base_cnn = create_model()

Input tensors: [<tf.Tensor 'right_arm_ipX:0' shape=(None, None, 3) dtype=float32>, <tf.Tensor 'right_wrist_ipX:0' shape=(None, None, 3) dtype=float32>, <tf.Tensor 'left_hip_ipX:0' shape=(None, None, 3) dtype=float32>, <tf.Tensor 'left_wrist_ipX:0' shape=(None, None, 3) dtype=float32>]


In [13]:
base_cnn.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
right_arm_ipX (InputLayer)      [(None, None, 3)]    0                                            
__________________________________________________________________________________________________
right_wrist_ipX (InputLayer)    [(None, None, 3)]    0                                            
__________________________________________________________________________________________________
left_hip_ipX (InputLayer)       [(None, None, 3)]    0                                            
__________________________________________________________________________________________________
left_wrist_ipX (InputLayer)     [(None, None, 3)]    0                                            
______________________________________________________________________________________________

In [14]:
from tensorflow.keras import backend as K

n_sensors = 4
n_filters = [3, 3, 3, 3] # X, Y, Z


# make the input layers
ips_l = []

n_f = 0
for sensor in all_sensors:
    ips_l.append(Input(shape = (None, n_filters[n_f]), name = sensor + '_ip_l'))
    n_f += 1
    
ips_r = []

n_f = 0
for sensor in all_sensors:
    ips_r.append(Input(shape = (None, n_filters[n_f]), name = sensor + '_ip_r'))
    n_f += 1

# encode each of the two inputs into a vector with the base conv model
encoded_l = base_cnn(ips_l)
encoded_r = base_cnn(ips_r)
# merge two encoded inputs with the l1 distance between them

diff = subtract([encoded_l,encoded_r])
l1_diff = K.abs(diff)
prediction = Dense(1, activation='sigmoid')(l1_diff)

siamese_net = Model([ips_l,ips_r], prediction)

optimizer = Adam(0.001)

siamese_net.compile(loss="binary_crossentropy",optimizer=optimizer)

siamese_net.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
right_arm_ip_l (InputLayer)     [(None, None, 3)]    0                                            
__________________________________________________________________________________________________
right_wrist_ip_l (InputLayer)   [(None, None, 3)]    0                                            
__________________________________________________________________________________________________
left_hip_ip_l (InputLayer)      [(None, None, 3)]    0                                            
__________________________________________________________________________________________________
left_wrist_ip_l (InputLayer)    [(None, None, 3)]    0                                            
____________________________________________________________________________________________

In [15]:
base_cnn.inputs

[<tf.Tensor 'right_arm_ipX:0' shape=(None, None, 3) dtype=float32>,
 <tf.Tensor 'right_wrist_ipX:0' shape=(None, None, 3) dtype=float32>,
 <tf.Tensor 'left_hip_ipX:0' shape=(None, None, 3) dtype=float32>,
 <tf.Tensor 'left_wrist_ipX:0' shape=(None, None, 3) dtype=float32>]

In [16]:
siamese_net.inputs

[<tf.Tensor 'right_arm_ip_l:0' shape=(None, None, 3) dtype=float32>,
 <tf.Tensor 'right_wrist_ip_l:0' shape=(None, None, 3) dtype=float32>,
 <tf.Tensor 'left_hip_ip_l:0' shape=(None, None, 3) dtype=float32>,
 <tf.Tensor 'left_wrist_ip_l:0' shape=(None, None, 3) dtype=float32>,
 <tf.Tensor 'right_arm_ip_r:0' shape=(None, None, 3) dtype=float32>,
 <tf.Tensor 'right_wrist_ip_r:0' shape=(None, None, 3) dtype=float32>,
 <tf.Tensor 'left_hip_ip_r:0' shape=(None, None, 3) dtype=float32>,
 <tf.Tensor 'left_wrist_ip_r:0' shape=(None, None, 3) dtype=float32>]

In [17]:
len(X[0])

80

In [18]:
len(X2[0])

105

In [19]:
np.array_equal(y_ohe[2], y_ohe[1])

False

In [20]:
np.array([1,2,3]).astype(np.float32)

array([1., 2., 3.], dtype=float32)

In [21]:
def triplet_generation(X1, y1, X2, y2): # pair generation
    X_m = []
    y_m = []
    for i in tqdm_notebook(range(len(X1[0]))):
        for j in range(len(X2[0])):
            x1 = [ X1[0][i], X1[1][i], X1[2][i], X1[3][i] ]
            x2 = [ X2[0][j], X2[1][j], X2[2][j], X2[3][j] ]
            
            left_i = [x1[0].reshape(1,x1[0].shape[0],3).astype(np.float32), 
              x1[1].reshape(1,x1[1].shape[0],3).astype(np.float32), 
              x1[2].reshape(1,x1[2].shape[0],3).astype(np.float32), 
              x1[3].reshape(1,x1[3].shape[0],3).astype(np.float32)]

            right_i = [x2[0].reshape(1,x2[0].shape[0],3).astype(np.float32), 
                      x2[1].reshape(1,x2[1].shape[0],3).astype(np.float32), 
                      x2[2].reshape(1,x2[2].shape[0],3).astype(np.float32), 
                      x2[3].reshape(1,x2[3].shape[0],3).astype(np.float32)]
            X_m.append(left_i + right_i)
            if np.array_equal(y1[i], y2[j]):
                y_m.append(1.0)
            else:
                y_m.append(0.0)
    return X_m, y_m

In [22]:
xx, yy = triplet_generation(X, y_ohe, X2, y2_ohe)

In [23]:
len(xx)

8400

In [24]:
xx10 = xx[10]

In [25]:
# len sanity check
print(len(xx10))
print(len(xx10[0]))
print(len(xx10[0][0]))
print(len(xx10[0][0][0]))

8
1
1399
3


In [26]:
print(xx10[0].shape)

(1, 1399, 3)


In [27]:
yy[10]

0.0

In [28]:
p = siamese_net.predict(xx10)
print(p)

[[0.6269838]]


In [29]:
siamese_net.train_on_batch(xx10, np.array(yy[10]).reshape(1,1))

12.879792

In [30]:
print(p)

[[0.6269838]]


In [31]:
# fixing empty arrays

for i in range(len(X)):
    for j in range(len(X[0])):
        if len(X) == 5 and i == len(X)-1: # assuming mocap always comes in the last index
            if len(X[i][j]) == 0:
                X[i][j] = np.zeros((1, 87))
        else:
            if len(X[i][j]) == 0:
                X[i][j] = np.array([[0, 0, 0]], dtype = np.float32)
                
for i in range(len(X2)):
    for j in range(len(X2[0])):
        if len(X2) == 5 and i == len(X2)-1: # assuming mocap always comes in the last index
            if len(X2[i][j]) == 0:
                X2[i][j] = np.zeros((1, 87))
        else:
            if len(X2[i][j]) == 0:
                X2[i][j] = np.array([[0, 0, 0]], dtype = np.float32)
                
                
for i in range(len(X3)):
    for j in range(len(X3[0])):
        if len(X3) == 5 and i == len(X3)-1: # assuming mocap always comes in the last index
            if len(X3[i][j]) == 0:
                X3[i][j] = np.zeros((1, 87))
        else:
            if len(X3[i][j]) == 0:
                X3[i][j] = np.array([[0, 0, 0]], dtype = np.float32)

In [32]:
def train(simple_conv, Xm, ym, EPOCH, train_split=1.0): # train on merged data
    # single zero padding for NULL datas

    report_acc = 0.0
    for epoch in range(EPOCH):
        print(f'Training epoch {epoch} ...')
        avg_loss = 0.0

        for sample_i in tqdm_notebook(range(5)): # replace later with len(Xm)
            x = Xm[sample_i]
            y_s = np.array(ym[sample_i])

            train_loss   =  simple_conv.train_on_batch(x, 
                                        y_s.reshape(1,1))
            avg_loss += train_loss/(len(Xm))

        print(f'loss: {avg_loss}')
    
    return simple_conv

In [33]:
aa = [X[0][5], X[1][5], X[2][5], X[3][5]]

In [34]:
len(aa)

4

In [35]:
len(aa[0])

1442

In [36]:
ab = aa + aa

In [37]:
len(ab)

8

In [38]:
len(ab[0])

1442

In [39]:
def process_x(x1, x2):
    return x1 + x2

In [40]:
np.argmax([0,1,0])

1

In [41]:
np.argmax([0,1,1])

1

In [42]:
def test(simple_conv, Xt, yt, Xs1, ys1, Xs2, ys2): # considering all samples for majority voting

    avg_acc = 0.0
    
    y_votes = []
    
    for sample_i in tqdm_notebook(range(5)): # len(Xt[0]) # Xt is in format (4,S,..) Xs1 and Xs2 are in same format too
        votes = [0, 0, 0] # initially all votes are zero
        for j in range(5): # len(Xs1[0])
            x1 = [Xt[0][sample_i], Xt[1][sample_i], Xt[2][sample_i], Xt[3][sample_i]]
            x2 = [Xs1[0][j], Xs1[1][j], Xs1[2][j], Xs1[3][j]]
            
            left_i = [x1[0].reshape(1,x1[0].shape[0],3).astype(np.float32), 
              x1[1].reshape(1,x1[1].shape[0],3).astype(np.float32), 
              x1[2].reshape(1,x1[2].shape[0],3).astype(np.float32), 
              x1[3].reshape(1,x1[3].shape[0],3).astype(np.float32)]

            right_i = [x2[0].reshape(1,x2[0].shape[0],3).astype(np.float32), 
                      x2[1].reshape(1,x2[1].shape[0],3).astype(np.float32), 
                      x2[2].reshape(1,x2[2].shape[0],3).astype(np.float32), 
                      x2[3].reshape(1,x2[3].shape[0],3).astype(np.float32)]
            
            xf = left_i + right_i
            p = simple_conv.predict(xf)[0][0]
            
            if p > 0.7: # only samples taken where I'm confident
                votes[ np.argmax(ys1[j]) ] += 1
            
            if p < 0.3: # I'm confident this is negative
                votes[ np.argmax(ys1[j]) ] -= 1
                
                
        for j in range(5): # len(Xs1[0])
            x1 = [Xt[0][sample_i], Xt[1][sample_i], Xt[2][sample_i], Xt[3][sample_i]]
            x2 = [Xs2[0][j], Xs2[1][j], Xs2[2][j], Xs2[3][j]]
            
            left_i = [x1[0].reshape(1,x1[0].shape[0],3).astype(np.float32), 
              x1[1].reshape(1,x1[1].shape[0],3).astype(np.float32), 
              x1[2].reshape(1,x1[2].shape[0],3).astype(np.float32), 
              x1[3].reshape(1,x1[3].shape[0],3).astype(np.float32)]

            right_i = [x2[0].reshape(1,x2[0].shape[0],3).astype(np.float32), 
                      x2[1].reshape(1,x2[1].shape[0],3).astype(np.float32), 
                      x2[2].reshape(1,x2[2].shape[0],3).astype(np.float32), 
                      x2[3].reshape(1,x2[3].shape[0],3).astype(np.float32)]
            
            xf = left_i + right_i

            p = simple_conv.predict(xf)[0][0]
            
            if p > 0.7: # only samples taken where I'm confident
                votes[ np.argmax(ys2[j]) ] += 1
            
            if p < 0.3: # I'm confident this is negative
                votes[ np.argmax(ys2[j]) ] -= 1
                
        y_votes.append(votes)
                
        y_p = np.array([0., 0., 0.])
        y_p[ np.argmax(votes) ] = 1.
        
        if np.array_equal(y_p, yt[sample_i]):
            avg_acc += 1./(len(Xt[0]))
                
    print(f'test acc: {avg_acc}')
    return avg_acc, y_votes

In [43]:
# train on subject1
# validate on subject2, 3

from tqdm import tqdm
from tensorflow.keras.optimizers import RMSprop

report = []

for xytst in [(X, y_ohe, 'X1'), (X2, y2_ohe, 'X2'), (X3, y3_ohe, 'X3')]:
    
    xytr = [a for a in [(X, y_ohe, 'X1'), (X2, y2_ohe, 'X2'), (X3, y3_ohe, 'X3')] if a!=xytst]

    EPOCH = 3
    train_split = 1.0
    lr = 0.001
    
    Xm, ym = triplet_generation(xytr[0][0], xytr[0][1], xytr[1][0], xytr[1][1])
    Xm2, ym2 = triplet_generation(xytr[0][0], xytr[0][1], xytr[0][0], xytr[0][1])
    Xm3, ym3 = triplet_generation(xytr[1][0], xytr[1][1], xytr[1][0], xytr[1][1])

    for e in range(EPOCH):
        siamese_net.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=lr))
        siamese_net = train(siamese_net, Xm, ym, 1, train_split)
        lr /= 1.2
        siamese_net.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=lr))
        siamese_net = train(siamese_net, Xm2, ym2, 1, train_split)
        lr /= 1.4
        siamese_net.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=lr))
        siamese_net = train(siamese_net, Xm3, ym3, 1, train_split)
        lr /= 1.5
    

    print('-------------------------------------------------')
    print('Test phase')
    print(f'Testing {xytst[2]}')

    t_acc, y_v = test(siamese_net, xytst[0], xytst[1], xytr[0][0], xytr[0][1], xytr[1][0], xytr[1][1])
    report.append(f'TS: {xytst[2]} test_acc: {t_acc}')
    print(y_v[:5])

print('--------------------report----------------------')
for ta in report:
    print(ta)


Training epoch 0 ...



loss: 0.0041796948366345935
Training epoch 0 ...



loss: 0.00405494365562387
Training epoch 0 ...



loss: 0.0041912032832231005
Training epoch 0 ...



loss: 0.0032898702281964247
Training epoch 0 ...


...


loss: 0.002953353284978542
Training epoch 0 ...



loss: 0.00360567535672869
Training epoch 0 ...



loss: 0.004673578888177872
Training epoch 0 ...



loss: 0.0028412786038284127
-------------------------------------------------
Test phase
Testing X3



test acc: 0.009708737864077669
[[3, 1, 5], [3, 2, 5], [3, 0, 4], [3, 2, 5], [3, 2, 4]]
--------------------report----------------------
TS: X1 test_acc: 0.025
TS: X2 test_acc: 0.009523809523809525
TS: X3 test_acc: 0.009708737864077669


In [44]:
5/80*0.5

0.03125